In [1]:
import numpy as np
import matplotlib.pyplot as plt
import oss
from scipy.interpolate import interp1d
from pathlib import Path
import math

In [2]:
class MassTrack:
    def __init__(data):
        global model
        ##self.data = np.genfromtxt(model, usecols = range(0, 61))
        
        ##self.get_ZAMS = get_ZAMS(self.data)
        ##self.get_IAMS = get_IAMS(self.data)
        ##self.get_TAMS = get_TAMS(self.data)
        ##self.get_RGBTip = get_RGBTip(self.data)
        ##self.get_sEEPs = get_sEEPs(self.data)
        
    def get_ZAMS(data):
        global ZAMS_array
        try:
            cut_array = data[data[:,28]> data[1,28] - 0.0015]
            Lfus = np.sum(cut_array[:, 13:17], axis = 1)
            Ltot = (10**(cut_array[:, 3]))* (3.846 * 10**33)
            LfusdivLtot = np.divide(Lfus, Ltot)
            ZAMS_age = cut_array[0:LfusdivLtot.argmax(), 2]
            LfusdivLtot_lim = LfusdivLtot[0:LfusdivLtot.argmax()]
            #plt.plot(LfusdivLtot_lim, ZAMS_age)

            ZAMS_x = LfusdivLtot_lim
            ZAMS_y = ZAMS_age

            int_curve = interp1d(ZAMS_x, ZAMS_y)
            interp_age = int_curve(0.999)
            int_curve_two = interp1d(data[:,2], data[:,0:], axis=0)
            ZAMS = np.array(int_curve_two(interp_age))
            return ZAMS

        except:
            central_temperature = data[:,23]
            ZAMS = data[central_temperature.argmax(),:]
            ##ZAMS_array = np.row_stack((ZAMS_array, ZAMS))
            return ZAMS

    def get_IAMS(data):
        global IAMS
        try:
            IAMS_x = data[:,28]
            IAMS_y = data[:,2]
            IAMS_int_curve = interp1d(IAMS_x, IAMS_y)
            IAMS_age = IAMS_int_curve(0.3)
            IAMS_int_curve_two = interp1d(data[:,2], data[:,0:], axis=0)
            IAMS = np.array(IAMS_int_curve_two(IAMS_age))
            return IAMS
            ##IAMS_array = np.row_stack((IAMS_array, IAMS))
            ##print("good")

        except:
            IAMS_error = np.zeros(shape = [1, 61])
            #IAMS_array = np.row_stack((IAMS_array, IAMS_error))
            #print("error")

    def get_TAMS(data):
        global TAMS_array
        try:
            TAMS_x = data[:,28]
            TAMS_y = data[:,2]
            TAMS_int_curve = interp1d(TAMS_x, TAMS_y)
            global TAMS_age 
            TAMS_age = TAMS_int_curve(10e-12)
            TAMS_int_curve_two = interp1d(data[:,2], data[:,0:], axis=0)
            TAMS = np.array(TAMS_int_curve_two(TAMS_age))
            return TAMS
            ##TAMS_array = np.row_stack((TAMS_array, TAMS))
        except:
            Col = data[0,:]
            ## TAMS_age set to 100 so that when TAMS_age is used to filter data in RGBTip_Function, the program errors and doesnt run for models that dont make it past the TAMS.
            TAMS_age = 100
            End = Col.argmax()
            TAMS = np.array(data[:,End])
            #TAMS_array = np.row_stack((TAMS_array, TAMS))

    def get_RGBTip(data):
        global RGBTip_array
        try:
            cut_array = data[data[:, 2] > TAMS_age]
            Lum = (10**(cut_array[:, 3]))* (3.846 * 10**33)
            Lum_Max = Lum.argmax()
            Teff = (10**(cut_array[:, 6]))
            Teff_Min = Teff.argmin()
            if (Teff_Min > Lum_Max):
                RGBTip = cut_array[Lum_Max,:]
                return RGBTip
                ##RGBTip_array = np.row_stack([RGBTip_array, RGBTip])
                ##Complete_files.append(file)
            else:
                RGBTip = cut_array[Teff_Min,:]
                return RGBTip
                ##RGBTip_array = np.row_stack([RGBTip_array, RGBTip])
                ##Complete_files.append(file)
        except:
            RGBTip_error = np.arange(60)
            RGBTip_error = RGBTip_error.fill(0)
            #RGBTip_array = np.vstack([RGBTip_array, RGBTip_error])

    def get_sEEPs(data):
        sEEp_ZAMS = (MassTrack.get_ZAMS(data))
        sEEp_IAMS = (MassTrack.get_IAMS(data))
        sEEp_TAMS = (MassTrack.get_TAMS(data))
        sEEp_RGBTip = (MassTrack.get_RGBTip(data))

        ZAMS_weight = math.sqrt((4.0*(sEEp_ZAMS[2])**2)+(1.25*(sEEp_ZAMS[6])**2)+(10*(sEEp_ZAMS[3])**2))
        IAMS_weight = ZAMS_weight + math.sqrt((4.0*(sEEp_IAMS[2]-sEEp_ZAMS[2])**2)+(1.25*(sEEp_IAMS[6]-sEEp_ZAMS[6])**2)+(10*(sEEp_IAMS[3]-sEEp_ZAMS[3])**2))
        TAMS_weight = IAMS_weight + math.sqrt((4.0*(sEEp_TAMS[2]-sEEp_IAMS[2])**2)+(1.25*(sEEp_TAMS[6]-sEEp_IAMS[6])**2)+(10*( sEEp_TAMS[3]-sEEp_IAMS[3])**2))
        RGBTip_weight = TAMS_weight + math.sqrt((4.0*(sEEp_RGBTip[2]-sEEp_TAMS[2])**2)+(1.25*(sEEp_RGBTip[6]-sEEp_TAMS[6])**2)+(10*(sEEp_RGBTip[3]-sEEp_TAMS[3])**2))

        b = data[data[:,2] > sEEp_ZAMS[2]]
        n=0
        point_weight = np.array([ZAMS_weight])
        
        for row in b:
            if(n < b.shape[0]-1):
                ##find the weight of all plotted points in the mass track
                weight = point_weight[n] + math.sqrt((4.0*(b[n+1,2]-b[n,2])**2)+(1.25*(b[n+1,6]-b[n,6])**2)+(10*(b[n+1,3]-b[n,3])**2))
                point_weight = np.row_stack([point_weight, weight])
                n = n+1
            else:
                n=n

        c = np.hstack((b,point_weight))

        Dzi = np.linspace(ZAMS_weight, IAMS_weight, 201)
        Dtr = np.linspace(TAMS_weight, RGBTip_weight, 201)
        Dit = np.linspace(IAMS_weight, TAMS_weight, 201)

        D_curve = interp1d(point_weight[:,0], c[:,0:], axis=0)
        
        properties_ZtoI = D_curve(Dzi)
        properties_ItoT = D_curve(Dit)
        properties_TtoR = D_curve(Dtr)

        return properties_ZtoI

In [3]:
class Isochrone():
    def __init__(self):
        age = input("What Age would you like the isochrone to be? ")
        
        def interpolate_ZAMS(age):
            global model
            ZAMSarray = np.empty(shape=[0, 61])
            for root, dirs, files in os.walk("./solar_models_test"):
                for file in files:
                    model = ('./solar_models_test/' + file)
                    ZAMS = MassTrack().get_ZAMS
                    ZAMSarray = np.row_stack((ZAMSarray, ZAMS))
            
            
            ##print(ZAMSarray)
            int_curve = interp1d(ZAMSarray[:,2], ZAMSarray[:,0:], axis=0)
            IsochroneZAMS = np.array(int_curve(age))
            return IsochroneZAMS
        
        def interpolate_IAMS(age):
            global model
            IAMSarray = np.empty(shape=[0, 61])
            for root, dirs, files in os.walk("./solar_models_test"):
                for file in files:
                    model = ('./solar_models_test/' + file)
                    IAMS = MassTrack().get_IAMS
                    try:
                        IAMSarray = np.row_stack((IAMSarray, IAMS))
                    except:
                        n=1
            
            
            int_curve = interp1d(IAMSarray[:,2], IAMSarray[:,0:], axis=0)
            IsochroneIAMS = np.array(int_curve(age))
            return IsochroneIAMS
        
        def interpolate_TAMS(age):
            global model
            TAMSarray = np.empty(shape=[0, 61])
            for root, dirs, files in os.walk("./solar_models_test"):
                for file in files:
                    model = ('./solar_models_test/' + file)
                    TAMS = MassTrack().get_TAMS
                    try:
                        TAMSarray = np.row_stack((TAMSarray, TAMS))
                    except:
                        n=1
            
            print(TAMSarray)
            int_curve = interp1d(TAMSarray[:,2], TAMSarray[:,0:], axis=0)
            IsochroneTAMS = np.array(int_curve(age))
            return IsochroneTAMS
        
        self.interpolate_ZAMS = interpolate_ZAMS(age)
        self.interpolate_IAMS = interpolate_IAMS(age)
        self.interpolate_TAMS = interpolate_TAMS(age)
        
        